[View in Colaboratory](https://colab.research.google.com/github/tabbydoc/table_detection/blob/master/table_detection.ipynb)

### Подключаем гуглдиск

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls "drive/Colab Notebooks/Tabby"

### Устанавливаем object detection API

In [0]:
%cd

In [0]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml

In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

In [0]:
import os
os.chdir('models/research')

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
import sys
sys.path.append('/content/models/research/slim')

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cp -r pycocotools <path_to_tensorflow>/models/research/

In [0]:
%run object_detection/builders/model_builder_test.py

### Загружаем модель

In [0]:
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!tar -xvf faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!cp faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.* ./models/research/

### Загружаем данные

In [0]:
!cp "drive/Colab Notebooks/Tabby/val.record" /root/models/research

In [0]:
!cp "drive/Colab Notebooks/Tabby/train.record" /root/models/research

In [0]:
!cp "drive/Colab Notebooks/Tabby/pipeline.config" /root/models/research

In [0]:
!cp -r "drive/Colab Notebooks/Tabby/annotations/label_map.pbtxt" /root/models/research/annotations/

In [0]:
%run object_detection/model_main.py --logtostderr --pipeline_config_path=pipeline.config --train_dir=train


In [0]:
%ls